<a href="https://colab.research.google.com/github/neildocs/InclusiveGatewayTestCase/blob/master/nlp/word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset

There is **TensorFlow Data Services / TFDS** contains many data sets for different categories.


One of these data set is **IMDB Review Data Set** which is available at http://ai.stanford.edu/~amaas/data/sentiment/

## Implementation

### Prerequisites

In [ ]:
import tensorflow as tf
print(tf.__version__)

# If running on TensorFlow v1.x, need run the below line
# tf.enable_eager_execution()

# If not using Colab, tensorflow_datasets requires be installed
# !pip install -q tensorflow_datasets

### Implementation

In [ ]:
# Import tfds
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [ ]:
# Split data set
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
# training
training_sentences = []
training_labels = []

for s, l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(str(l.numpy()))


In [ ]:
# testing
testing_sentences = []
testing_labels = []

for s, l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(str(l.numpy()))


In [ ]:
training_labels_final = np.array(training_labels).astype("int")
testing_labels_final = np.array(testing_labels).astype("int")

In [ ]:
# Tokenize sentences

vocabulary_size = 10000
embedding_dim = 16  # 16-dimentional array for vectors
max_length = 120
trunc_type = "post"
oov_token = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocabulary_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [ ]:
# -------------
# Define model
# -------------

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Embedding(vocabulary_size, embedding_dim, input_length = max_length),
    keras.layers.Flatten(),  # or use 'GlobalAveragePooling1D'
    keras.layers.Dense(6, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(loss = "binary_crossentropy", 
              optimizer = "adam", 
              metrics = ["accuracy"])
model.summary()

In [ ]:
# ---------
# Training 
# ---------
num_epochs = 10
model.fit(padded, 
          training_labels_final, 
          epochs = num_epochs, 
          validation_data = (testing_padded, testing_labels_final))

In [ ]:
# ----------------------------
# Analyze the embedding layer
# ----------------------------
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape)    # shape: (vocabulary_size, embedding_dim)

In [ ]:
# ---------------------------------------------------------------------
# To plot it, 
# 
# As it currently stands, our word index has the key being the word, 
# and the value being the token for the word. 
# 
# We'll need to flip this around, to look through the padded list 
# to decode the tokens back into the words
# ---------------------------------------------------------------------
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [ ]:
# ---------------------------------------------------------
# Write the vectors and their metadata to files.
# TensorFlow use the files to plot the vectors in 3D space
# ---------------------------------------------------------
import io

out_v = io.open('vecs.tsv', 'w', encoding = 'utf-8')    # vector
out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')    # metadata

for word_num in range(1, vocabulary_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + '\n')
    # To the vectors file, we simply write out the value of each of the items 
    # in the array of embeddings, i.e, the co-efficient of each dimension 
    # on the vector for this word.
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()

In [ ]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

Use the downloaded files, and render the results on [TensorFlow Embedding Projector](https://projector.tensorflow.org)